In [1]:
from __future__ import annotations
try:
    from mediocreatbest import auto, run
except ImportError:
    %pip install --user --upgrade pip
    %pip install --user --upgrade --force-reinstall \
        git+https://gist.github.com/player1537/3457b026ed6ef6696d758517f55a58df#egg=mediocreatbest[jupyter]
    from mediocreatbest import auto, run


In [25]:
!du -ch /mnt/seenas2/data/_backup_sinai/home/thobson2/src/GraphShaders/examples/NBER-Patents/data/*.bin

181M	/mnt/seenas2/data/_backup_sinai/home/thobson2/src/GraphShaders/examples/NBER-Patents/data/NBER-Patents.edge.element.2u32.bin
13M	/mnt/seenas2/data/_backup_sinai/home/thobson2/src/GraphShaders/examples/NBER-Patents/data/NBER-Patents.node.appyear.u32.bin
13M	/mnt/seenas2/data/_backup_sinai/home/thobson2/src/GraphShaders/examples/NBER-Patents/data/NBER-Patents.node.cat1.u32.bin
13M	/mnt/seenas2/data/_backup_sinai/home/thobson2/src/GraphShaders/examples/NBER-Patents/data/NBER-Patents.node.cat2.u32.bin
13M	/mnt/seenas2/data/_backup_sinai/home/thobson2/src/GraphShaders/examples/NBER-Patents/data/NBER-Patents.node.gotyear.u32.bin
13M	/mnt/seenas2/data/_backup_sinai/home/thobson2/src/GraphShaders/examples/NBER-Patents/data/NBER-Patents.node.x.f32.bin
13M	/mnt/seenas2/data/_backup_sinai/home/thobson2/src/GraphShaders/examples/NBER-Patents/data/NBER-Patents.node.y.f32.bin
253M	total


In [28]:
@run(after=lambda x: print(f'array(shape={x.shape!r}, dtype={x.dtype!r})'))
def edge():
    home = auto.pathlib.Path('/mnt/seenas2/data/_backup_sinai/home/thobson2')
    
    return auto.numpy.memmap(
        home / 'src/GraphShaders/examples/NBER-Patents/data/NBER-Patents.edge.element.2u32.bin',
        dtype=[ ('i', 'u4'), ('j', 'u4') ],
    )

array(shape=(23644705,), dtype=dtype([('i', '<u4'), ('j', '<u4')]))


In [26]:
@run(after=lambda x: print(f'array(shape={x.shape!r}, dtype={x.dtype!r})'))
def xpos():
    home = auto.pathlib.Path('/mnt/seenas2/data/_backup_sinai/home/thobson2')
    
    return auto.numpy.memmap(
        home / 'src/GraphShaders/examples/NBER-Patents/data/NBER-Patents.node.x.f32.bin',
        dtype=[ ('x', 'f4') ],
    )

array(shape=(3149407,), dtype=dtype([('x', '<f4')]))


In [27]:
@run(after=lambda x: print(f'array(shape={x.shape!r}, dtype={x.dtype!r})'))
def ypos():
    home = auto.pathlib.Path('/mnt/seenas2/data/_backup_sinai/home/thobson2')
    
    return auto.numpy.memmap(
        home / 'src/GraphShaders/examples/NBER-Patents/data/NBER-Patents.node.y.f32.bin',
        dtype=[ ('y', 'f4') ],
    )

array(shape=(3149407,), dtype=dtype([('y', '<f4')]))


In [33]:
@run(after=print)
def xdom(xpos, /):
    xmin = xpos['x'].min()
    xmax = xpos['x'].max()
    return xmin, xmax

@run(after=print)
def ydom(ypos, /):
    ymin = ypos['y'].min()
    ymax = ypos['y'].max()
    return ymin, ymax


(0.0, 1.0)
(0.0, 1.0)


In [ ]:
@run
def __test_filter(edge, xpos, ypos, /):
#     edge = edge[:10]
    
    box = auto.numpy.zeros(
        shape=(1,),
        dtype=[ ('xlo', 'f4'), ('ylo', 'f4'), ('xhi', 'f4'), ('yhi', 'f4') ],
    )
    box['xlo'][0] = 0.25
    box['ylo'][0] = 0.25
    box['xhi'][0] = 0.50
    box['yhi'][0] = 0.50
    
    def shape(shape: tuple[int, ...], y: auto.numpy.NDArray):
        assert isinstance(shape, tuple)
        assert shape == y.shape, \
            f"""Expected {shape=!r}; Got shape={y.shape!r}"""
        return y
    
    
    #---
    
    assert auto.numpy.all(box['xlo'] < box['xhi'])
    assert auto.numpy.all(box['ylo'] < box['yhi'])
    
    NV = xpos.shape[0]
    NE = edge.shape[0]
    NB = box.shape[0]
    
    xgt = shape((NV, NB),   xpos['x'][:, None] >= box['xlo'][None, :])
    xlt = shape((NV, NB),   xpos['x'][:, None] <= box['xhi'][None, :])
    ygt = shape((NV, NB),   ypos['y'][:, None] >= box['ylo'][None, :])
    ylt = shape((NV, NB),   ypos['y'][:, None] <= box['yhi'][None, :])
    
    within = shape((NV, NB),   (xgt) & (xlt) & (ygt) & (ylt))
    
    ix = shape((NE,),   xpos['x'][edge['i']])
    iy = shape((NE,),   ypos['y'][edge['i']])
    jx = shape((NE,),   xpos['x'][edge['j']])
    jy = shape((NE,),   ypos['y'][edge['j']])
    iwithin = shape((NE, NB),   within[edge['i'], :])
    jwithin = shape((NE, NB),   within[edge['j'], :])
    
    vertical = shape((NE,),   auto.numpy.isclose(ix, jx))
    horizontal = shape((NE,),   auto.numpy.isclose(iy, jy))
    dy = shape((NE,),   auto.numpy.subtract(jy, iy))
    dx = shape((NE,),   auto.numpy.subtract(jx, ix))
    slope = shape((NE,),   auto.numpy.divide(dy, dx))
    
    # Solve [ y - y0 = m * (x - x0) ] for [ x ] and [ y ]
    #   x = (y - y0) / m + x0
    #   y = m * (x - x0) + y0
    nx = shape((NE, NB),   (box['ylo'][None, :] - iy[:, None]) / slope[:, None] + ix[:, None])
    ey = shape((NE, NB),   (box['xhi'][None, :] - ix[:, None]) * slope[:, None] + iy[:, None])
    sx = shape((NE, NB),   (box['yhi'][None, :] - iy[:, None]) / slope[:, None] + ix[:, None])
    wy = shape((NE, NB),   (box['xlo'][None, :] - ix[:, None]) * slope[:, None] + ix[:, None])
    
    ninter = shape((NE, NB),   (box['xlo'] <= nx) & (nx <= box['xhi']))
    einter = shape((NE, NB),   (box['ylo'] <= ey) & (ey <= box['yhi']))
    sinter = shape((NE, NB),   (box['xlo'] <= sx) & (sx <= box['xhi']))
    winter = shape((NE, NB),   (box['ylo'] <= wy) & (wy <= box['yhi']))
    
    inter = shape((NE, NB),   ninter | einter | sinter | winter)
    
    keep = shape((NE, NB),   iwithin | jwithin | inter)
    
    partitions: list[auto.numpy.ndarray] = []
    edge = auto.numpy.lib.recfunctions.structured_to_unstructured(
        edge,
        dtype='u4',
    )
    for i in range(NB):
        auto.numpy; import numpy.lib.recfunctions
        partition = auto.numpy.select(keep[:, i], edge)
        partition = auto.numpy.lib.recfunctions.unstructured_to_structured(
            partition,
            dtype=[ ('i', 'u4'), ('j', 'u4') ],
        )
        
        print(f'box {i}: {partition.shape=!r}')
    
    return partitions
        

/tmp/ipykernel_13509/510893039.py:48: RuntimeWarning: divide by zero encountered in divide
  slope = shape((NE,),   auto.numpy.divide(dy, dx))
/tmp/ipykernel_13509/510893039.py:48: RuntimeWarning: invalid value encountered in divide
  slope = shape((NE,),   auto.numpy.divide(dy, dx))
/tmp/ipykernel_13509/510893039.py:53: RuntimeWarning: divide by zero encountered in divide
  nx = shape((NE, NB),   (box['ylo'][None, :] - iy[:, None]) / slope[:, None] + ix[:, None])
/tmp/ipykernel_13509/510893039.py:55: RuntimeWarning: divide by zero encountered in divide
  sx = shape((NE, NB),   (box['yhi'][None, :] - iy[:, None]) / slope[:, None] + ix[:, None])
